In [3]:
import paddle 
import paddle.nn as nn
import paddle.nn.functional as F
class ConvBNLayer(paddle.nn.Layer):
    def __init__(self,numchannals,num_fileters,filter_size,stride=1,groups=1,act=None):
        super(ConvBNLayer,self).__init__()
        
        self.conv = nn.Conv2D(
        in_channels=numchannals,
        out_channels=num_fileters,
        kernel_size=filter_size,
        stride=stride,
        padding=(filter_size-1)//2,
        groups=groups,
        bias_attr = False)
        self.batch_norm = nn.BatchNorm2D(num_fileters)
        self.act = act
        
    def forward(self,inputs):
        y = self.conv(inputs)
        y = self.batch_norm(y)
        if self.act =='leaky':
            y=F.leaky_relu(x=y,negative_slope=0.1)
        elif self.act =='relu':
            y=F.relu(y)
        return y

In [4]:
import paddle
import paddle.nn.functional as F
class BottleneckBlock(paddle.nn.Layer):
    def __init__(self,num_channels,num_filters,stride,shortcut=True):
        super(BottleneckBlock, self).__init__()
        # 创建第一个卷积层 1x1
        self.conv0 = ConvBNLayer(
            numchannals=num_channels,
            num_fileters=num_filters,
            filter_size=1,
            act='relu')

        # 创建第二个卷积层 3x3

        self.conv1 = ConvBNLayer(
            numchannals=num_filters,
            num_fileters=num_filters,
            filter_size=3,
            stride=stride,
            act='relu')
        # 创建第三个卷积 1x1，但输出通道数乘以4
        self.conv2 = ConvBNLayer(
            numchannals=num_filters,
            num_fileters=num_filters * 4,
            filter_size=1,
            act=None)
        # 如果conv2的输出跟此残差块的输入数据形状一致，则shortcut=True
        # 否则shortcut = False，添加1个1x1的卷积作用在输入数据上，使其形状变成跟conv2一致
        if not shortcut:
            self.short = ConvBNLayer(
                numchannals=num_channels,
                num_fileters=num_filters * 4,
                filter_size=1,
                stride=stride)
        self.shortcut = shortcut
        self._num_channels_out = num_filters * 4
    def forward(self, inputs):
        y = self.conv0(inputs)
        conv1 = self.conv1(y)
        conv2 = self.conv2(conv1)
        # 如果shortcut=True，直接将inputs跟conv2的输出相加
        # 否则需要对inputs进行一次卷积，将形状调整成跟conv2输出一致
        if self.shortcut:
            short = inputs
        else:
            short = self.short(inputs)
        y = paddle.add(x=short, y=conv2)
        y = F.relu(y)
        return y

# resnet

In [5]:
# 定义ResNet模型
import numpy as np
import paddle
import paddle.nn as nn
import paddle.nn.functional as F
class ResNet(paddle.nn.Layer):
    def __init__(self, layers=50, class_dim=1):
        """
        layers, 网络层数，可以是50, 101或者152
        class_dim，分类标签的类别数
        """
        super(ResNet, self).__init__()
        self.layers = layers
        supported_layers = [50, 101, 152]
        assert layers in supported_layers, \
"supported layers are {} but input layer is {}".format(supported_layers, layers)
        if layers == 50:
            #ResNet50包含多个模块，其中第2到第5个模块分别包含3、4、6、3个残差块
            depth = [3, 4, 6, 3]
        elif layers == 101:
            #ResNet101包含多个模块，其中第2到第5个模块分别包含3、4、23、3个残差块
            depth = [3, 4, 23, 3]
        elif layers == 152:
            #ResNet152包含多个模块，其中第2到第5个模块分别包含3、8、36、3个残差块
            depth = [3, 8, 36, 3]
        # 残差块中使用到的卷积的输出通道数
        num_filters = [64, 128, 256, 512]
        # ResNet的第一个模块，包含1个7x7卷积，后面跟着1个最大池化层
        self.conv = ConvBNLayer(
            numchannals=3,
            num_fileters=64,
            filter_size=7,
            stride=2,
            act='relu')
        self.pool2d_max = nn.MaxPool2D(
            kernel_size=3,
            stride=2,
            padding=1)
        # ResNet的第二到第五个模块c2、c3、c4、c5
        self.bottleneck_block_list = []
        num_channels = 64
        for block in range(len(depth)):
            shortcut = False
            for i in range(depth[block]):
                # c3、c4、c5将会在第一个残差块使用stride=2；其余所有残差块stride=1
                bottleneck_block = self.add_sublayer(
                    'bb_%d_%d' % (block, i),
                    BottleneckBlock(
                        num_channels=num_channels,
                        num_filters=num_filters[block],
                        stride=2 if i == 0 and block != 0 else 1, 
                        shortcut=shortcut))
                num_channels = bottleneck_block._num_channels_out
                self.bottleneck_block_list.append(bottleneck_block)
                shortcut = True
        # 在c5的输出特征图上使用全局池化
        self.pool2d_avg = paddle.nn.AdaptiveAvgPool2D(output_size=1)
        # stdv用来作为全连接层随机初始化参数的方差
        import math
        stdv = 1.0 / math.sqrt(2048 * 1.0)
        # 创建全连接层，输出大小为类别数目，经过残差网络的卷积和全局池化后，
        # 卷积特征的维度是[B,2048,1,1]，故最后一层全连接的输入维度是2048
        self.out = nn.Linear(in_features=2048, out_features=class_dim,
                      weight_attr=paddle.ParamAttr(
                          initializer=paddle.nn.initializer.Uniform(-stdv, stdv)))
    def forward(self, inputs):
        y = self.conv(inputs)
        y = self.pool2d_max(y)
        for bottleneck_block in self.bottleneck_block_list:
            y = bottleneck_block(y)
        y = self.pool2d_avg(y)
        y = paddle.reshape(y, [y.shape[0], -1])
        y = self.out(y)
        return y

In [6]:
model_1 = ResNet()
print(model_1)

ResNet(
  (conv): ConvBNLayer(
    (conv): Conv2D(3, 64, kernel_size=[7, 7], stride=[2, 2], padding=3, data_format=NCHW)
    (batch_norm): BatchNorm2D(num_features=64, momentum=0.9, epsilon=1e-05)
  )
  (pool2d_max): MaxPool2D(kernel_size=3, stride=2, padding=1)
  (bb_0_0): BottleneckBlock(
    (conv0): ConvBNLayer(
      (conv): Conv2D(64, 64, kernel_size=[1, 1], data_format=NCHW)
      (batch_norm): BatchNorm2D(num_features=64, momentum=0.9, epsilon=1e-05)
    )
    (conv1): ConvBNLayer(
      (conv): Conv2D(64, 64, kernel_size=[3, 3], padding=1, data_format=NCHW)
      (batch_norm): BatchNorm2D(num_features=64, momentum=0.9, epsilon=1e-05)
    )
    (conv2): ConvBNLayer(
      (conv): Conv2D(64, 256, kernel_size=[1, 1], data_format=NCHW)
      (batch_norm): BatchNorm2D(num_features=256, momentum=0.9, epsilon=1e-05)
    )
    (short): ConvBNLayer(
      (conv): Conv2D(64, 256, kernel_size=[1, 1], data_format=NCHW)
      (batch_norm): BatchNorm2D(num_features=256, momentum=0.9, epsilo

In [7]:
from paddle.vision import resnet50

model = resnet50()
x = paddle.rand([1,3,224,224])
out = model(x)
print(out.shape)

D:\conda\envs\pytorch\lib\site-packages\paddle\nn\layer\norm.py:824: UserWarning: When training, we now always track global mean and variance.
  warnings.warn(


[1, 1000]


In [8]:
from paddle.vision.models import resnet50
from paddle.vision.datasets import Cifar10
from paddle.optimizer import Momentum
from paddle.regularizer import L2Decay
from paddle.nn import CrossEntropyLoss
from paddle.metric import Accuracy
from paddle.vision.transforms import Transpose
paddle.vision.set_image_backend('cv2')
model = paddle.Model(resnet50(pretrained=False, num_classes=10))
train_dataset = Cifar10(mode = 'train',transform=Transpose())
val_dataset = Cifar10(mode='test',transform=Transpose())

optimizer = Momentum(learning_rate=0.01, momentum=0.9,weight_decay=L2Decay(1e-4),parameters=model.parameters())

model.prepare(optimizer,CrossEntropyLoss(),Accuracy(topk=(1,5)))

model.fit(train_dataset,val_dataset,epochs=50,batch_size=64,save_dir='./output',num_workers=8)

D:\conda\envs\pytorch\lib\site-packages\paddle\io\reader.py:429: UserWarning: DataLoader with multi-process mode is not supported on MacOs and Windows currently. Please use signle-process mode with num_workers = 0 instead
  warnings.warn(


The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/50
step  10/782 - loss: 7.9059 - acc_top1: 0.1094 - acc_top5: 0.4891 - 121ms/step
step  20/782 - loss: 4.3738 - acc_top1: 0.1297 - acc_top5: 0.5461 - 86ms/step
step  30/782 - loss: 6.7132 - acc_top1: 0.1318 - acc_top5: 0.5672 - 74ms/step
step  40/782 - loss: 4.7281 - acc_top1: 0.1277 - acc_top5: 0.5797 - 68ms/step
step  50/782 - loss: 9.5035 - acc_top1: 0.1313 - acc_top5: 0.5813 - 65ms/step
step  60/782 - loss: 5.3554 - acc_top1: 0.1354 - acc_top5: 0.5813 - 62ms/step
step  70/782 - loss: 4.5363 - acc_top1: 0.1366 - acc_top5: 0.5900 - 61ms/step
step  80/782 - loss: 4.6638 - acc_top1: 0.1473 - acc_top5: 0.6057 - 59ms/step
step  90/782 - loss: 3.8496 - acc_top1: 0.1550 - acc_top5: 0.6160 - 58ms/step
step 100/782 - loss: 7.2787 - acc_top1: 0.1581 - acc_top5: 0.6272 - 57ms/step
step 110/782 - loss: 4.0366 - acc_top1: 0.1631 - acc_top5: 0.6357 - 57ms/step
step 120/782 - loss:

D:\conda\envs\pytorch\lib\site-packages\paddle\nn\layer\norm.py:824: UserWarning: When training, we now always track global mean and variance.
  warnings.warn(


step  10/782 - loss: 2.1821 - acc_top1: 0.1781 - acc_top5: 0.6094 - 53ms/step
step  20/782 - loss: 3.8519 - acc_top1: 0.1625 - acc_top5: 0.6227 - 52ms/step
step  30/782 - loss: 3.7383 - acc_top1: 0.1667 - acc_top5: 0.6286 - 51ms/step
step  40/782 - loss: 2.4963 - acc_top1: 0.1797 - acc_top5: 0.6484 - 51ms/step
step  50/782 - loss: 2.9330 - acc_top1: 0.1888 - acc_top5: 0.6516 - 51ms/step
step  60/782 - loss: 2.8296 - acc_top1: 0.1906 - acc_top5: 0.6667 - 51ms/step
step  70/782 - loss: 2.5877 - acc_top1: 0.1897 - acc_top5: 0.6708 - 50ms/step
step  80/782 - loss: 3.0268 - acc_top1: 0.1936 - acc_top5: 0.6762 - 50ms/step
step  90/782 - loss: 2.1350 - acc_top1: 0.1970 - acc_top5: 0.6819 - 50ms/step
step 100/782 - loss: 3.9354 - acc_top1: 0.1988 - acc_top5: 0.6881 - 50ms/step
step 110/782 - loss: 2.9908 - acc_top1: 0.2024 - acc_top5: 0.6928 - 51ms/step
step 120/782 - loss: 4.1277 - acc_top1: 0.2044 - acc_top5: 0.6951 - 51ms/step
step 130/782 - loss: 2.7165 - acc_top1: 0.2061 - acc_top5: 0.695

D:\conda\envs\pytorch\lib\site-packages\paddle\nn\layer\norm.py:824: UserWarning: When training, we now always track global mean and variance.
  warnings.warn(


...

step  10/782 - loss: 0.1635 - acc_top1: 0.9594 - acc_top5: 0.9969 - 51ms/step
step  20/782 - loss: 0.2202 - acc_top1: 0.9602 - acc_top5: 0.9977 - 50ms/step
step  30/782 - loss: 0.1541 - acc_top1: 0.9536 - acc_top5: 0.9969 - 50ms/step
step  40/782 - loss: 0.2274 - acc_top1: 0.9520 - acc_top5: 0.9973 - 50ms/step
step  50/782 - loss: 0.1201 - acc_top1: 0.9522 - acc_top5: 0.9978 - 50ms/step
step  60/782 - loss: 0.1605 - acc_top1: 0.9516 - acc_top5: 0.9977 - 50ms/step
step  70/782 - loss: 0.0705 - acc_top1: 0.9533 - acc_top5: 0.9978 - 50ms/step
step  80/782 - loss: 0.1304 - acc_top1: 0.9539 - acc_top5: 0.9977 - 50ms/step
step  90/782 - loss: 0.1501 - acc_top1: 0.9556 - acc_top5: 0.9979 - 50ms/step
step 100/782 - loss: 0.1016 - acc_top1: 0.9564 - acc_top5: 0.9981 - 50ms/step
step 110/782 - loss: 0.0670 - acc_top1: 0.9578 - acc_top5: 0.9983 - 50ms/step
step 120/782 - loss: 0.0335 - acc_top1: 0.9582 - acc_top5: 0.9983 - 50ms/step
step 130/782 - loss: 0.0690 - acc_top1: 0.9593 - acc_top5: 0.998

D:\conda\envs\pytorch\lib\site-packages\paddle\nn\layer\norm.py:824: UserWarning: When training, we now always track global mean and variance.
  warnings.warn(


step  10/782 - loss: 0.1214 - acc_top1: 0.9547 - acc_top5: 0.9984 - 52ms/step
step  20/782 - loss: 0.2399 - acc_top1: 0.9555 - acc_top5: 0.9977 - 51ms/step
step  30/782 - loss: 0.0913 - acc_top1: 0.9568 - acc_top5: 0.9984 - 51ms/step
step  40/782 - loss: 0.1101 - acc_top1: 0.9578 - acc_top5: 0.9988 - 51ms/step
step  50/782 - loss: 0.1304 - acc_top1: 0.9603 - acc_top5: 0.9988 - 50ms/step
step  60/782 - loss: 0.1786 - acc_top1: 0.9591 - acc_top5: 0.9990 - 50ms/step
step  70/782 - loss: 0.2315 - acc_top1: 0.9585 - acc_top5: 0.9987 - 51ms/step
step  80/782 - loss: 0.0289 - acc_top1: 0.9592 - acc_top5: 0.9986 - 51ms/step
step  90/782 - loss: 0.0439 - acc_top1: 0.9613 - acc_top5: 0.9988 - 51ms/step
step 100/782 - loss: 0.1453 - acc_top1: 0.9608 - acc_top5: 0.9986 - 51ms/step
step 110/782 - loss: 0.2494 - acc_top1: 0.9612 - acc_top5: 0.9984 - 51ms/step
step 120/782 - loss: 0.1395 - acc_top1: 0.9615 - acc_top5: 0.9986 - 51ms/step
step 130/782 - loss: 0.1980 - acc_top1: 0.9625 - acc_top5: 0.998

D:\conda\envs\pytorch\lib\site-packages\paddle\nn\layer\norm.py:824: UserWarning: When training, we now always track global mean and variance.
  warnings.warn(


step  10/782 - loss: 0.2251 - acc_top1: 0.9500 - acc_top5: 1.0000 - 53ms/step
step  20/782 - loss: 0.3331 - acc_top1: 0.9430 - acc_top5: 0.9984 - 53ms/step
step  30/782 - loss: 0.1279 - acc_top1: 0.9464 - acc_top5: 0.9979 - 53ms/step
step  40/782 - loss: 0.2004 - acc_top1: 0.9492 - acc_top5: 0.9977 - 52ms/step
step  50/782 - loss: 0.0921 - acc_top1: 0.9494 - acc_top5: 0.9981 - 52ms/step
step  60/782 - loss: 0.0790 - acc_top1: 0.9513 - acc_top5: 0.9984 - 52ms/step
step  70/782 - loss: 0.0608 - acc_top1: 0.9542 - acc_top5: 0.9987 - 52ms/step
step  80/782 - loss: 0.2009 - acc_top1: 0.9563 - acc_top5: 0.9986 - 51ms/step
step  90/782 - loss: 0.1554 - acc_top1: 0.9568 - acc_top5: 0.9986 - 51ms/step
step 100/782 - loss: 0.0359 - acc_top1: 0.9578 - acc_top5: 0.9988 - 51ms/step
step 110/782 - loss: 0.0964 - acc_top1: 0.9592 - acc_top5: 0.9989 - 51ms/step
step 120/782 - loss: 0.0892 - acc_top1: 0.9605 - acc_top5: 0.9988 - 52ms/step
step 130/782 - loss: 0.2581 - acc_top1: 0.9609 - acc_top5: 0.998